### Page Scrapper

In [23]:
import requests
import pandas as pd

In [3]:
link = 'https://quotes.toscrape.com/'

In [4]:
# Page is imported : You got the response from the website successfully
res = requests.get(link)

In [5]:
# Returns the html code for the web page
html = res.text

In [6]:
type(html)

str

In [7]:
# Create a file in write mode and save html code into it
fd = open('main.html','w')
fd.write(html)
fd.close()

### Scraping Quotes

In [8]:
from bs4 import BeautifulSoup # Helps in scraping

In [9]:
res = requests.get('https://quotes.toscrape.com/')
# Soup : Object containing the whole string of data
soup = BeautifulSoup(res.text,'html.parser')

In [10]:
soup

<!DOCTYPE html>

<html lang="en">
<head>
<meta charset="utf-8"/>
<title>Quotes to Scrape</title>
<link href="/static/bootstrap.min.css" rel="stylesheet"/>
<link href="/static/main.css" rel="stylesheet"/>
</head>
<body>
<div class="container">
<div class="row header-box">
<div class="col-md-8">
<h1>
<a href="/" style="text-decoration: none">Quotes to Scrape</a>
</h1>
</div>
<div class="col-md-4">
<p>
<a href="/login">Login</a>
</p>
</div>
</div>
<div class="row">
<div class="col-md-8">
<div class="quote" itemscope="" itemtype="http://schema.org/CreativeWork">
<span class="text" itemprop="text">“The world as we have created it is a process of our thinking. It cannot be changed without changing our thinking.”</span>
<span>by <small class="author" itemprop="author">Albert Einstein</small>
<a href="/author/Albert-Einstein">(about)</a>
</span>
<div class="tags">
            Tags:
            <meta class="keywords" content="change,deep-thoughts,thinking,world" itemprop="keywords"/>
<a class="

In [11]:
# Extract text from soup whose class='text' and tag='span'
# Slicing : Removing 1st and last string
print(soup.find('span', class_='text').text[1:-1])

The world as we have created it is a process of our thinking. It cannot be changed without changing our thinking.


In [12]:
quotes =[]
for quote in soup.find_all('span',class_='text'):
    quotes.append(quote.text[1:-1])

In [13]:
quotes

['The world as we have created it is a process of our thinking. It cannot be changed without changing our thinking.',
 'It is our choices, Harry, that show what we truly are, far more than our abilities.',
 'There are only two ways to live your life. One is as though nothing is a miracle. The other is as though everything is a miracle.',
 'The person, be it gentleman or lady, who has not pleasure in a good novel, must be intolerably stupid.',
 "Imperfection is beauty, madness is genius and it's better to be absolutely ridiculous than absolutely boring.",
 'Try not to become a man of success. Rather become a man of value.',
 'It is better to be hated for what you are than to be loved for what you are not.',
 "I have not failed. I've just found 10,000 ways that won't work.",
 "A woman is like a tea bag; you never know how strong it is until it's in hot water.",
 'A day without sunshine is like, you know, night.']

### Scraping Quotes with Author Details

In [14]:
len(soup.find_all('div',class_='quote'))

10

In [15]:
# Extracting 1st quote data
soup.find_all('div',class_='quote')[0]

<div class="quote" itemscope="" itemtype="http://schema.org/CreativeWork">
<span class="text" itemprop="text">“The world as we have created it is a process of our thinking. It cannot be changed without changing our thinking.”</span>
<span>by <small class="author" itemprop="author">Albert Einstein</small>
<a href="/author/Albert-Einstein">(about)</a>
</span>
<div class="tags">
            Tags:
            <meta class="keywords" content="change,deep-thoughts,thinking,world" itemprop="keywords"/>
<a class="tag" href="/tag/change/page/1/">change</a>
<a class="tag" href="/tag/deep-thoughts/page/1/">deep-thoughts</a>
<a class="tag" href="/tag/thinking/page/1/">thinking</a>
<a class="tag" href="/tag/world/page/1/">world</a>
</div>
</div>

In [19]:
data = []
# Extracting each quote
for sp in soup.find_all('div',class_ = 'quote'):
    quote = sp.find('span',class_ = 'text').text[1:-1]
    # Extracting name of the author
    author = sp.find('small',class_ = 'author').text
    # Extracting author_id of each author
    author_id = sp.find('a').get('href')
    # Extracting all tags for each quote
    tags = []
    for tag in sp.find_all('a',class_='tag'):
        tags.append(tag.text)
        
    tags = ','.join(tags)
    data.append((quote,author,author_id,tags))

In [20]:
len(data)

10

In [22]:
data[0][1]

'Albert Einstein'

In [25]:
df = pd.DataFrame(data,columns=['quote','author','author_id','tags'])

In [28]:
df

,quote,author,author_id,tags
0,The world as we have created it is a process o...,Albert Einstein,/author/Albert-Einstein,"change,deep-thoughts,thinking,world"
1,"It is our choices, Harry, that show what we tr...",J.K. Rowling,/author/J-K-Rowling,"abilities,choices"
2,There are only two ways to live your life. One...,Albert Einstein,/author/Albert-Einstein,"inspirational,life,live,miracle,miracles"
3,"The person, be it gentleman or lady, who has n...",Jane Austen,/author/Jane-Austen,"aliteracy,books,classic,humor"
4,"Imperfection is beauty, madness is genius and ...",Marilyn Monroe,/author/Marilyn-Monroe,"be-yourself,inspirational"
5,Try not to become a man of success. Rather bec...,Albert Einstein,/author/Albert-Einstein,"adulthood,success,value"
6,It is better to be hated for what you are than...,André Gide,/author/Andre-Gide,"life,love"
7,"I have not failed. I've just found 10,000 ways...",Thomas A. Edison,/author/Thomas-A-Edison,"edison,failure,inspirational,paraphrased"
8,A woman is like a tea bag; you never know how ...,Eleanor Roosevelt,/author/Eleanor-Roosevelt,misattributed-eleanor-roosevelt
9,"A day without sunshine is like, you know, night.",Steve Martin,/author/Steve-Martin,"humor,obvious,simile"


In [27]:
df.to_csv('Quotes.csv',index=False)

### Scraping Quotes from Multiple Pages

In [32]:
# Gives progress of percentage of work done
from tqdm import tqdm
data = []
for page in tqdm(range(1,11)):
    link = 'https://quotes.toscrape.com/page/'+ str(page)
    # Getting response
    res = requests.get(link)
    soup = BeautifulSoup(res.text,'html.parser')
    
    # Extracting each quote
    for sp in soup.find_all('div',class_ = 'quote'):
        quote = sp.find('span',class_ = 'text').text[1:-1]
        # Extracting name of the author
        author = sp.find('small',class_ = 'author').text
        # Extracting author_id of each author
        author_id = sp.find('a').get('href')
        # Extracting all tags for each quote
        tags = []
        for tag in sp.find_all('a',class_='tag'):
            tags.append(tag.text)

        tags = ','.join(tags)
        data.append((quote,author,author_id,tags))

100%|███████████████████████████████████████████| 10/10 [00:22<00:00,  2.21s/it]


In [33]:
len(data)

100

In [34]:
df = pd.DataFrame(data,columns=['quote','author','author_id','tags'])

In [42]:
df.head()

,quote,author,author_id,tags,author_link
0,The world as we have created it is a process o...,Albert Einstein,/author/Albert-Einstein,"change,deep-thoughts,thinking,world",https://quotes.toscrape.com/author/Albert-Eins...
1,"It is our choices, Harry, that show what we tr...",J.K. Rowling,/author/J-K-Rowling,"abilities,choices",https://quotes.toscrape.com/author/J-K-Rowling
2,There are only two ways to live your life. One...,Albert Einstein,/author/Albert-Einstein,"inspirational,life,live,miracle,miracles",https://quotes.toscrape.com/author/Albert-Eins...
3,"The person, be it gentleman or lady, who has n...",Jane Austen,/author/Jane-Austen,"aliteracy,books,classic,humor",https://quotes.toscrape.com/author/Jane-Austen
4,"Imperfection is beauty, madness is genius and ...",Marilyn Monroe,/author/Marilyn-Monroe,"be-yourself,inspirational",https://quotes.toscrape.com/author/Marilyn-Monroe


In [38]:
df['author_link'] = 'https://quotes.toscrape.com' + df['author_id']

In [40]:
df.head()

,quote,author,author_id,tags,author_link
0,The world as we have created it is a process o...,Albert Einstein,/author/Albert-Einstein,"change,deep-thoughts,thinking,world",https://quotes.toscrape.com/author/Albert-Eins...
1,"It is our choices, Harry, that show what we tr...",J.K. Rowling,/author/J-K-Rowling,"abilities,choices",https://quotes.toscrape.com/author/J-K-Rowling
2,There are only two ways to live your life. One...,Albert Einstein,/author/Albert-Einstein,"inspirational,life,live,miracle,miracles",https://quotes.toscrape.com/author/Albert-Eins...
3,"The person, be it gentleman or lady, who has n...",Jane Austen,/author/Jane-Austen,"aliteracy,books,classic,humor",https://quotes.toscrape.com/author/Jane-Austen
4,"Imperfection is beauty, madness is genius and ...",Marilyn Monroe,/author/Marilyn-Monroe,"be-yourself,inspirational",https://quotes.toscrape.com/author/Marilyn-Monroe


In [41]:
df.to_csv('100Quotes.csv',index=False)